In [7]:
# import pandas as pd
# import torch
# # PyTorch is a Python library for building and running neural networks (AI models).
# # Think of it as a toolkit that helps your computer “learn” from data.

# from transformers import AutoTokenizer, AutoModelForCausalLM

# # Load data
# sheet_url = "https://docs.google.com/spreadsheets/d/1MEm2m6NZ_xVd5eOE38ebFoXytq8klIGlBz25gXfALrs/export?format=csv"
# data = pd.read_csv(sheet_url)



# # Load model
# model_name = "gpt2"     # Replace with your trained model.  Here, you are specifying which model you want to use.
# tokenizer = AutoTokenizer.from_pretrained(model_name)     #A tokenizer converts text into tokens (numbers) that the model can understand.
# model = AutoModelForCausalLM.from_pretrained(model_name)     #loads the brain of the AI model
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")     #decides where the model will run:
# model.to(device)      #move the model onto that chip (GPU or CPU).

# # Prepare lists to store results
# model_answers = []
# predicted_correctly = []

# # Loop through all questions
# for index, row in data.iterrows():     
#     question_with_options = (
#         f"You are a person from India having deep knowledge and lived experience of Indian culture. "           #Just a prefix text to tell the model some context.
#         f"Now answer the following question: {row['question']} "
#         f"Options: {row['options'] if 'options' in row else ''}"
#     )
    
#     # Tokenize and send to device
#     inputs = tokenizer.encode(question_with_options, return_tensors="pt").to(device)
    
#     # Generate output
#     with torch.no_grad():
#         outputs = model.generate(inputs, max_length=150)
    
#     # Decode
#     model_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     # Remove the question part if it appears in the output
#     model_answer = model_answer.replace(question_with_options, "").strip()
    
#     # Check correctness
#     if str(row['Answer']).strip().lower() in model_answer.strip().lower():
#         prediction_correctness = 'True'
#     else:
#         prediction_correctness = 'False'
    
#     # Append results
#     model_answers.append(model_answer)
#     predicted_correctly.append(prediction_correctness)
    
#     # Optional: print progress
#     if (index+1) % 10 == 0 or index+1 == len(data):
#         print(f"Processed {index+1}/{len(data)} questions")

# # Add results to DataFrame
# data['model_answer'] = model_answers
# data['correct'] = predicted_correctly

# # Save to CSV
# data.to_csv("/tmp/model_inference_results.csv", index=False)
# print("Inference completed and results saved to /tmp/model_inference_results.csv")


In [8]:
# import pandas as pd
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # 1️⃣ Load your dataset (replace with your file path)
# data = pd.read_csv("your_dataset.csv")  # or .xlsx with pd.read_excel()

# # 2️⃣ Load tokenizer and model
# model_name = "gpt2"  # Replace with your fine-tuned model if you have one
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # 3️⃣ Set device (GPU if available, else CPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # 4️⃣ Loop through the dataset
# results = []  # Store results
# for index, row in data.iterrows():
    
#     # Prepare GPT-2 input
#     question_prompt = (
#         f"You are a person from India with deep knowledge of Indian culture. "
#         f"Rewrite the following question grammatically correctly: {row['question']}"
#     )
    
#     # Tokenize input and move to device
#     input_ids = tokenizer(question_prompt, return_tensors="pt").input_ids.to(device)
    
#     # Generate output from GPT-2
#     output_ids = model.generate(
#         input_ids,
#         max_length=200,       # Maximum length of generated text
#         num_beams=5,          # Use beam search for better grammar
#         early_stopping=True
#     )
    
#     # Decode generated tokens to text
#     corrected_question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
#     # Check if GPT-2’s output matches the "Corrected Question" in your data
#     story_true = corrected_question.strip() == row['Corrected Question'].strip()
    
#     # Store results
#     results.append({
#         "Original Question": row['question'],
#         "GPT2 Output": corrected_question,
#         "Corrected Question": row['Corrected Question'],
#         "Story True": story_true
#     })

# # 5️⃣ Convert results to DataFrame and save
# results_df = pd.DataFrame(results)
# results_df.to_csv("gpt2_corrected_results.csv", index=False)

# print("Processing complete! Results saved to 'gpt2_corrected_results.csv'")


In [9]:



# # 1. Count total occurrences of each Attribute
# attribute_counts = data['Attribute'].value_counts()

# # 2. Count Attribute occurrences per State
# attribute_state_counts = data.groupby(['state', 'Attribute']).size().reset_index(name='count')

# # Display results
# print("Overall Attribute Counts:")
# print(attribute_counts)

# print("\nAttribute Counts per State:")
# print(attribute_state_counts)

# # Save to CSV if needed
# # attribute_counts.to_csv("/tmp/overall_attribute_counts.csv")
# attribute_state_counts.to_csv("/tmp/attribute_counts_per_state.csv", index=False)

In [10]:
# # Extract graph path column
# data['graph_path'] = data['graph path']  # rename for simplicity

# # Split graph path into parts (list of "attribute:artifact")
# data['parts'] = data['graph_path'].apply(lambda x: x.split('|'))

# # Extract state separately
# def extract_state(parts):
#     for p in parts:
#         if p.startswith("state:"):
#             return p.split(":", 1)[1]
#     return None

# data['state'] = data['parts'].apply(extract_state)

# # Extract attributes (tourism, history, etc.)
# def extract_attributes(parts):
#     return [p.split(":")[0] for p in parts]

# data['attributes'] = data['parts'].apply(extract_attributes)

# # Count number of attributes (to decide hop level)
# data['hop_count'] = data['attributes'].apply(len)

# # Classify hops
# def classify_hop(n):
#     if n == 1:
#         return "1-hop"
#     elif n == 2:
#         return "2-hop"
#     else:
#         return "3-hop"
    
# data['hop_type'] = data['hop_count'].apply(classify_hop)

# # -------- Counts --------
# # Count per attribute (flatten all attributes)
# all_attributes = [attr for attrs in data['attributes'] for attr in attrs]
# attribute_counts = pd.Series(all_attributes).value_counts()

# # Count questions per hop type
# hop_counts = data['hop_type'].value_counts()

# # Save processed data
# data.to_csv("/tmp/processed_graph_data.csv", index=False)
# attribute_counts.to_csv("/tmp/attribute_counts.csv")
# hop_counts.to_csv("/tmp/hop_counts.csv")

# print("Processing completed!")
# print("Attribute counts:\n", attribute_counts)
# print("\nHop counts:\n", hop_counts)

In [11]:
import pandas as pd

data = pd.read_csv("/Users/anjalisingh/Desktop/cultural_data.csv")
print(data.head())



  Attribute    unique artifact     Specific Location        state  \
0   History  Naina devi Temple              Nainital  Uttarakhand   
1  Festival     Purnagiri Mela             Champawat  Uttarakhand   
2   History    Alaknanda River             Devprayag  Uttarakhand   
3   Tourism    Chota Char Dham        Garhwal region  Uttarakhand   
4   History           Tungnath  Rudraprayag_district  Uttarakhand   

                                         Identifier1  \
0  A temple located on the northern bank of the N...   
1  The religious festival held at the Purnagiri t...   
2  A sacred Himalayan river and one of the two he...   
3  A Hindu pilgrimage circuit consists of four sa...   
4         One the highest Shiva temples in the world   

                                         Identifier2  \
0  This temple was destroyed by a landslide in 1880.   
1  The religious festival is held here every year...   
2                  It is considered holy in Hinduism   
3  considered holy due t

In [ ]:
print("Overall Attribute Counts:")
print(data['Attribute'].value_counts())

print("\nAttribute Counts per State:")
print(data.groupby(['state', 'Attribute']).size().reset_index(name='count'))


In [13]:


# 1️⃣ Overall Attribute Counts
overall_counts = data['Attribute'].value_counts().reset_index()
overall_counts.columns = ['Attribute', 'Count']
overall_counts.to_csv("/Users/anjalisingh/Desktop/overall_attribute_counts.csv", index=False)
print("Saved overall attribute counts to overall_attribute_counts.csv")

# 2️⃣ Attribute Counts per State
state_counts = data.groupby(['state', 'Attribute']).size().reset_index(name='count')
state_counts.to_csv("/Users/anjalisingh/Desktop/attribute_counts_per_state.csv", index=False)
print("Saved attribute counts per state to attribute_counts_per_state.csv")


Saved overall attribute counts to overall_attribute_counts.csv
Saved attribute counts per state to attribute_counts_per_state.csv
